---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

print(nx.__version__)


1.11


---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def deg_dist(G):
    deg = G.degree()
    deg_vals = sorted(set(deg.values()))
    
    return [list(deg.values()).count(i) / float(nx.number_of_nodes(G)) for i in deg_vals ]

In [4]:
for i in range(len(P1_Graphs)):
    print('P1_Graphs', i)
    print(deg_dist(P1_Graphs[i]))
    print()

P1_Graphs 0
[0.465, 0.237, 0.1, 0.056, 0.032, 0.026, 0.016, 0.014, 0.009, 0.007, 0.005, 0.005, 0.002, 0.001, 0.001, 0.002, 0.002, 0.003, 0.002, 0.001, 0.002, 0.002, 0.002, 0.001, 0.002, 0.001, 0.002, 0.001, 0.001]

P1_Graphs 1
[0.002, 0.023, 0.166, 0.615, 0.171, 0.022, 0.001]

P1_Graphs 2
[0.0026666666666666666, 0.124, 0.756, 0.10533333333333333, 0.012]

P1_Graphs 3
[0.3453333333333333, 0.17733333333333334, 0.11866666666666667, 0.07733333333333334, 0.06266666666666666, 0.03333333333333333, 0.025333333333333333, 0.017333333333333333, 0.02, 0.018666666666666668, 0.010666666666666666, 0.009333333333333334, 0.006666666666666667, 0.012, 0.005333333333333333, 0.0013333333333333333, 0.0013333333333333333, 0.005333333333333333, 0.008, 0.0013333333333333333, 0.0026666666666666666, 0.0013333333333333333, 0.008, 0.004, 0.0013333333333333333, 0.0026666666666666666, 0.0026666666666666666, 0.0013333333333333333, 0.0026666666666666666, 0.0026666666666666666, 0.004, 0.0026666666666666666, 0.0013333333

In [5]:
def graph_identification():
    
    # Your Code Here
    ans = []
    
    for G in P1_Graphs:
        avg_clu = nx.average_clustering(G)
        avg_shr_pat = nx.average_shortest_path_length(G)
        deg_dis = deg_dist(G)
        
        if len(deg_dis) > 10:
            ans.append('PA')
        elif avg_clu < 0.1:
            ans.append('SW_H')
        else:
            ans.append('SW_L')

    return ans

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

def management_salary(n):
    managementSalary = n[1]['ManagementSalary']
    if managementSalary == 0:
        return 0
    elif managementSalary == 1:
        return 1
    else:
        return None

In [8]:
def salary_predictions():
    
    # Your Code Here
    print('dsa')
    
    # Making the df as basis for model training
    df = pd.DataFrame(index=G.nodes())
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G,normalized=True))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['pagerank'] = pd.Series(nx.pagerank(G))
    
    df['management'] = pd.Series([management_salary(n) for n in G.nodes(data=True)])
    
    # Prepare the train and test data sets
    df_train = df[~pd.isnull(df['management'])]
    df_test = df[pd.isnull(df['management'])]
    
    X_train = df_train[['betweenness', 'clustering', 'closeness', 'degree', 'degree_centrality', 'pagerank']]
    Y_train = df_train['management']
    X_test = df_test[['betweenness', 'clustering', 'closeness', 'degree', 'degree_centrality', 'pagerank']]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit the model
    clf = MLPClassifier(hidden_layer_sizes=[10,5], activation='relu', solver='lbfgs', alpha=1, random_state = 0)
    clf.fit(X_train_scaled, Y_train)
    
    pred = clf.predict_proba(X_test_scaled)[:, 1]
    ans = pd.Series(pred, X_test.index)

    return ans

salary_predictions()

dsa


1       0.022871
2       0.764009
5       0.999329
8       0.175283
14      0.197309
18      0.102368
27      0.265034
30      0.440463
31      0.131973
34      0.087776
37      0.012557
40      0.037150
45      0.108600
54      0.674314
55      0.923862
60      0.073123
62      0.999132
65      0.995341
77      0.023877
79      0.019454
97      0.028719
101     0.008184
103     0.284367
108     0.025339
113     0.212232
122     0.001455
141     0.485505
142     0.797604
144     0.014549
145     0.785182
          ...   
913     0.000572
914     0.007027
915     0.000266
918     0.131348
923     0.000028
926     0.014471
931     0.001844
934     0.000023
939     0.000008
944     0.000510
945     0.000118
947     0.109781
950     0.020079
951     0.000085
953     0.000419
959     0.000669
962     0.000559
963     0.117630
968     0.009594
969     0.009916
974     0.023928
984     0.000015
987     0.035595
989     0.012567
991     0.018185
992     0.001054
994     0.000708
996     0.0005

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [9]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [22]:
def new_connections_predictions():
    
    # Your Code Here
    
    # Making the preferential attachment df
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    pre_att = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(n[0],n[1]) for n in pre_att])
    df['preferential_attachment'] = [n[2] for n in pre_att]
    
    # Making the cn_soundarajan_hopcroft df
    cn_s_h = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_s_h = pd.DataFrame(index = [(n[0], n[1]) for n in cn_s_h])
    df_cn_s_h['cn_soundarajan_hopcroft'] = [n[2] for n in cn_s_h]
    
    # combine both dfs and future connections
    df = df.join(df_cn_s_h, how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df = future_connections.join(df, how='outer')
    
    # add RAI and JC
    df['resource_allocation_index'] = [n[2] for n in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [n[2] for n in list(nx.jaccard_coefficient(G))]
    
    # Prepare the train and test data sets
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    
    X_train = df_train[['preferential_attachment', 'cn_soundarajan_hopcroft', 'resource_allocation_index', 'jaccard_coefficient']]
    Y_train = df_train['Future Connection']
    X_test = df_test[['preferential_attachment', 'cn_soundarajan_hopcroft', 'resource_allocation_index', 'jaccard_coefficient']]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Fit the model
    clf = MLPClassifier(hidden_layer_sizes=[10,5], activation='relu', solver='lbfgs', alpha=1, random_state = 0)
    clf.fit(X_train_scaled, Y_train)
    
    pred = clf.predict_proba(X_test_scaled)[:, 1]
    ans = pd.Series(pred, X_test.index)
    
    return ans

new_connections_predictions()

(0, 9)          0.027397
(0, 19)         0.058384
(0, 20)         0.243534
(0, 35)         0.012674
(0, 38)         0.008163
(0, 42)         0.841010
(0, 43)         0.019892
(0, 50)         0.010742
(0, 53)         0.544904
(0, 54)         0.042026
(0, 62)         0.293897
(0, 63)         0.034352
(0, 69)         0.020840
(0, 72)         0.011663
(0, 83)         0.137357
(0, 90)         0.030040
(0, 91)         0.016633
(0, 95)         0.123267
(0, 100)        0.027070
(0, 106)        0.971702
(0, 108)        0.014786
(0, 109)        0.011987
(0, 110)        0.011348
(0, 118)        0.009365
(0, 122)        0.030919
(0, 131)        0.038419
(0, 133)        0.775436
(0, 140)        0.077755
(0, 149)        0.157343
(0, 151)        0.008238
                  ...   
(988, 989)      0.012526
(988, 996)      0.012542
(988, 997)      0.060991
(988, 1000)     0.012516
(988, 1002)     0.012542
(989, 994)      0.012536
(989, 996)      0.012536
(989, 1003)     0.012536
(989, 1004)     0.012536
